<a href="https://colab.research.google.com/github/aureliowozhiak/facebook_prophet_covid19/blob/main/notebooks/Facebook_Prophet_e_os_casos_de_COVID_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

<p align="center"><img src="https://raw.githubusercontent.com/aureliowozhiak/facebook_prophet_covid19/main/images/prophet_logo.png" /></p>

# Facebook Prophet e os casos de COVID-19

## Análisando a série temporal dos casos de Corona Virus no Brasil e criando previsões<sup>*</sup> com o Facebook Prophet

<sup><sub>*As previsões criadas aqui pelo Prophet não são exatas pois não mostram de fato o cenário futuro do corona virus, pois os dados utilizados apenas nos dão uma visão, com base no que aconteceu no passado, para termos uma noção de qual poderia ser a tendência de crescimento dos casos, porém, como os dados não exprimem  o cenário completo e não leva em conta campanhas futuras de vacinação, que já estão acontecendo, não devemos levar essas conclusões como se fossem verdades absolutas.</sub></sup>


---
## Visão geral do projeto:

 - Objetivo do projeto
 - Introdução
    - O movimento antivacina
    - Por que o SUS é importante?
        - Programa Nacional de Imunizações (PNI)
    - O Brasil em dados libertos
 - Importação de bibliotecas
 - Origem dos dados e informações técnicas
 - Carregamento e Processamento dos dados
 - Análisando a série temporal dos casos de COVID-19 no Brasil
 - Criando o modelo de previsão
 - Plotando as previsões com o Prophet<sup>*</sup>
 - Pesquisa complementar
 - Conclusão
 - Referências